<a href="https://colab.research.google.com/github/mweiher15/CollectiveIntelCapstone/blob/main/COMP440_ExtractingDataInPAWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We used the code below in PAWS notebook to extract the Wikipedia page assessments. The code was provided by Morten Warncke-Wang and Isaac Johnson.

In [ ]:
from datetime import datetime, timedelta
import time

import pandas as pd
import pymysql
import requests
from scipy.stats import spearmanr
import seaborn as sns

In [ ]:
days_worth_of_data = 28 

In [ ]:
# Groundtruth data is held in MariaDB replica databases accessible via PAWS

def make_connection(wikidb):
    """Connects to a host and database of the same name."""
    return pymysql.connect(
        host=f"{wikidb}.analytics.db.svc.wikimedia.cloud",
        read_default_file="~/.my.cnf",
        database=f"{wikidb}_p",
        charset='utf8'
    )

def query(conn, query):
    """Execute a SQL query against the connection, and return **all** the results."""
    with conn.cursor() as cur:
        cur.execute(query)
        data = cur.fetchall()
        return data
    
def get_assessments(wikidb):
    conn = make_connection(wikidb)
    results = query(conn, groundtruth_query)
    conn.close()
    return results

def assessments_to_df(results, wikidb):
    df = pd.DataFrame(results, columns=['page_id', 'revid', 'quality'])
    print(f'{len(df)} rows to start for {wikidb}.')
    df = df.sort_values(by=['page_id', 'revid'])
    df = df.drop_duplicates(subset='page_id', keep="last")  # keep most recent assessment
    print(f'{len(df)} rows after dropping duplicate pages.')
    df['quality'] = df['quality'].apply(lambda x: x.decode('utf-8'))
    if wikidb == 'frwiki':
        df['enqual'] = df['quality'].apply(frqual_to_enqual)
    elif wikidb == 'arwiki':
        df['enqual'] = df['quality'].apply(arqual_to_enqual)
    elif wikidb == 'trwiki':
        df['enqual'] = df['quality'].apply(trqual_to_enqual)
    elif wikidb == 'huwiki':
        df['enqual'] = df['quality'].apply(huqual_to_enqual)
    else:
        df['enqual'] = df['quality'].apply(lambda x: x if x in enqual_categories else None)
    df = df[~pd.isnull(df['enqual'])]
    print(f'{len(df)} rows after removing unknown/missing ratings.')
    df['qual_float'] = df['enqual'].apply(enqual_to_float)
    print("\nSample:")
    print(df.head(10))
    print("\nQual distribution:")
    print(df['enqual'].value_counts())
    df['wiki_db'] = wikidb
    return df

# get groundtruth assessments from the previous X days
four_weeks_ago = datetime.today() - timedelta(days=days_worth_of_data)
start_ts = f'{four_weeks_ago.year}{str(four_weeks_ago.month).rjust(2, "0")}{str(four_weeks_ago.day).rjust(2, "0")}000000'

groundtruth_query = f"""
SELECT
  pa_page_id AS page_id,
  pa_page_revision AS rev_id,
  pa_class AS qual_score
  
FROM page_assessments
INNER JOIN revision
  ON (pa_page_revision = rev_id)
WHERE
  rev_timestamp > {start_ts}
"""

#  pa_page_id in (305, 624, 765) AND
print(groundtruth_query)

In [ ]:
# A-class are left out because there are so few
# Descriptions: https://en.wikipedia.org/wiki/Wikipedia:Content_assessment
enqual_categories = ['Stub', 'Start', 'C' , 'B', 'GA', 'FA']

# functions for converting between quality classes and scores in different languages
def float_to_enqual(qual_score):
    # set based on:
    # en_sample.groupby('enqual')['pred_qual_float'].median().reindex(enqual_categories)
    # and equivalent for fr/ar with a little tweaking to create more separation
    # and keep labels within correct group (start/stub; c/b; ga/fa)
    if qual_score <= 0.36:
        return 'Stub'
    elif qual_score <= 0.54:
        return 'Start'
    elif qual_score <= 0.65:
        return 'C'
    elif qual_score <= 0.78:
        return 'B'
    elif qual_score <= 0.88:
        return 'GA'
    elif qual_score <= 1:
        return 'FA'
    else:
        return None

def enqual_to_float(enqual):
    # place-holder float values useful for computing rank correlations
    # for better estimates, see `float_to_enqual` above
    if enqual == 'FA':
        return 6/6
    elif enqual == 'GA':
        return 5/6
    elif enqual == 'B':
        return 4/6
    elif enqual == 'C':
        return 3/6
    elif enqual == 'Start':
        return 2/6
    elif enqual == 'Stub':
        return 1/6
    else:
        return None

    
def stratify_sample(df, sample_size=500, min_per_class=50):
    """Build stratified sample from quality groundtruth.
    
    Constraints:
    * at least {min_per_class} samples per class (where possible)
    * {sample_size} total rows
    """
    df = df.sample(frac=1)  # random order
    df['keep'] = False
    rows_per_qual = df['enqual'].value_counts().to_dict()
    for qc in rows_per_qual:
        rows_per_qual[qc] = min(min_per_class, rows_per_qual[qc])
    additional_samples = sample_size - sum(rows_per_qual.values())
    for i, row in enumerate(df.itertuples()):
        if rows_per_qual[row.enqual] > 0:
            df.loc[row.Index, 'keep'] = True
            rows_per_qual[row.enqual] = rows_per_qual[row.enqual] - 1
        elif additional_samples > 0:
            df.loc[row.Index, 'keep'] = True
            additional_samples -= 1
    df = df[df['keep']]
    df = df.drop(columns='keep')
    return df

In [ ]:
en_df.to_csv("page_assessments.csv", index=False)

In [ ]:
stub_df = en_df.loc[en_df["quality"] == "Stub"]
stub_df.to_csv("stub_assessments.csv",index=False)

We used the following code to get the name of the page assosiated with Wikipedia ID and get the average page news

In [ ]:
# Makes the plots appear within the notebook
%matplotlib inline

# Two fundamental packages for doing data manipulation
import numpy as np                   # http://www.numpy.org/
import pandas as pd                  # http://pandas.pydata.org/

from datetime import datetime
from datetime import timedelta

# Two related packages for plotting data
import matplotlib.pyplot as plt      # http://matplotlib.org/
import seaborn as sb                 # https://stanford.edu/~mwaskom/software/seaborn/

# Package for requesting data via the web and parsing resulting JSON
import requests                      # http://docs.python-requests.org/en/master/
import json                          # https://docs.python.org/3/library/json.html
from bs4 import BeautifulSoup        # https://www.crummy.com/software/BeautifulSoup/bs4/doc/

# Two packages for accessing the MySQL server
import pymysql                       # http://pymysql.readthedocs.io/en/latest/
import os                            # https://docs.python.org/3.4/library/os.html

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}



# Setup the code environment to use plots with a white background and DataFrames show more columns and rows
sb.set_style('whitegrid')
pd.options.display.max_columns = 100
pd.options.display.max_rows = 110

In [ ]:
page_title = "Dog"

In [ ]:
# Get today's date and yesterday's date
today = datetime.today()
yesterday = today - timedelta(days = 1)

# Convert to strings
today_s = datetime.strftime(today,'%Y%m%d00')
yesterday_s = datetime.strftime(yesterday,'%Y%m%d00')

# Get the pageviews for today and yesterday
url_string = 'http://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/{0}/daily/{1}/{2}'
print(url_string.format(page_title.replace(' ','_'),yesterday_s,today_s))

In [ ]:
def get_daily_pageviews(page_title):
    url_string = 'http://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/{0}/daily/2015010100/{1}'
    today = datetime.strftime(datetime.today(),'%Y%m%d00')
    req = requests.get(url_string.format(page_title,today), headers=headers)
    #return req.text
    json_s = json.loads(req.text)
    if 'items' in json_s.keys():
        _df = pd.DataFrame(json_s['items'])[['timestamp','views','article']]
        _df['timestamp'] = pd.to_datetime(_df['timestamp'],format='%Y%m%d00')
        _df['weekday'] = _df['timestamp'].apply(lambda x:x.weekday())
        return int(_df['views'].mean())

In [ ]:
titles_df = pd.read_csv("stub_titles.csv")
titles_df.head()

In [ ]:
titles_df['average_views']=titles_df['article_title'].apply(get_daily_pageviews)
titles_df.to_csv("stub_pageviews.csv", index=False)

In [ ]:
pageview_df = get_daily_pageviews(page_title)
print(pageview_df)
#pageview_df['timestamp'][2844].day
int(pageview_df['views'].mean())
# pageview_df.head()

In [ ]:
import requests
import json

def getTitle(id):
    # Define the API endpoint URL
    url = 'https://en.wikipedia.org/w/api.php'

    # Define the parameters for the API call
    params = {
        'action': 'query',
        'prop': 'info',
        'inprop': 'url',
        'pageids': id,  # replace with the article ID number
        'format': 'json'
    }

    # Make the API call and parse the response
    response = requests.get(url, params=params, headers=headers)
    data = json.loads(response.text)

    # Get the article title from the response
    if 'query' in data and 'pages' in data['query']:
        pages = data['query']['pages']
        for page_id, page_data in pages.items():
            if 'title' in page_data:
                title = page_data['title']
                return title
            else:
                return "N/A"


In [ ]:
stub_df = pd.read_csv("stub_assessments.csv")

In [ ]:
stub_df['article_title']=stub_df['page_id'].apply(getTitle)
stub_df.to_csv("stub_titles.csv", index=False)

In [ ]:
head = stub_df.loc[:4]
head

In [ ]:
head['article_title']=head['page_id'].apply(getTitle)

In [ ]:
head.head